In [38]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
#import matplotlib.pyplot as plt
import tensorflow as tf
import time
from pickle import load

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

%matplotlib inline
#import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow.keras.backend as K

from pickle import dump

from numpy import array
from numpy import hstack
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
from tensorflow.keras.layers import Dense,Flatten,Dropout, MaxPooling1D

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping


from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion

import toml

from math import floor



In [451]:

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = 0
        value_max = 2*np.pi
        minmax.append([value_min, value_max])
    return minmax
 
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
    return dataset
 

In [452]:
def load_df(path): 
    df = pd.read_csv(path, index_col=None)

    return df

In [453]:
def get_csv_files(dataset_path, keyword):
    """
    Generator function to recursively output the CSV files in a directory and its sub-directories.
    Arguments:
        dataset_path: Path to the directory containing the CSV files.
    Outputs:
        Paths of the found CSV files.
    """
    files = []
 
    for f in glob.glob(os.path.join(dataset_path,"*{}*".format(keyword))):
        #print(f)
        if not os.path.isdir(f):
            file_name, extension = f.split('.')            
            if extension == "csv":
                files.append(f)
            else:
                logging.warn("Invalid file: {}. Ignoring...".format(f))

    return files

In [454]:
def split_sequences(sequences, n_steps, LAT, n_in_seq, n_out_seq):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        # find the end of this pattern
        stop_ix = i + n_steps
        shift_ix = i + n_steps + LAT
        seq_length = len(sequences) - LAT
        if stop_ix > seq_length:
          break
        seq_x = sequences[i:stop_ix,:-n_in_seq]
        seq_y = sequences[shift_ix-1, -n_out_seq:]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)
    #return X, y

In [469]:
###########################################
# Preprocessing steps: 
# 1.) Euler(deg) 
# 2.) Euler(deg) + |Euler(deg).min|%360 
# 3.) Euler(deg:0-360) into Euler(rad:0-2pi) 
# 4.) Scale between: 0-1 
##############################################

def get_sequence_euler_radiant(dataframe, dataset_path_train):
    # multivariate data preparation

    # define sequences
    in_seq4 = dataframe[['pitch']].to_numpy()
    in_seq5 = dataframe[['roll']].to_numpy()
    in_seq6 = dataframe[['yaw']].to_numpy()
    
    in_seq4_minimum = np.min(in_seq4)
    in_seq5_minimum = np.min(in_seq5)
    in_seq6_minimum = np.min(in_seq6)

    out_seq4 = dataframe[['pitch']].to_numpy()
    out_seq5 = dataframe[['roll']].to_numpy()
    out_seq6 = dataframe[['yaw']].to_numpy()

    print("Original y_true : " + str(in_seq4[19]))
    
    out_seq4_minimum = np.min(out_seq4)
    out_seq5_minimum = np.min(out_seq5)
    out_seq6_minimum = np.min(out_seq6)
    
    in_seq4 = (in_seq4 + np.abs(in_seq4_minimum))%360
    in_seq5 = (in_seq5 + np.abs(in_seq5_minimum))%360
    in_seq6 = (in_seq6 + np.abs(in_seq6_minimum))%360
    
    out_seq4 = (out_seq4 + np.abs(out_seq4_minimum))%360
    out_seq5 = (out_seq5 + np.abs(out_seq5_minimum))%360
    out_seq6 = (out_seq6 + np.abs(out_seq6_minimum))%360
    
     # transform into radiant
    in_seq4 = np.deg2rad(in_seq4)
    in_seq5 = np.deg2rad(in_seq5)
    in_seq6 = np.deg2rad(in_seq6)
    
    out_seq4 = np.deg2rad(out_seq4)
    out_seq5 = np.deg2rad(out_seq5)
    out_seq6 = np.deg2rad(out_seq6)    
    
    print("Shifted rad y_true : " + str(in_seq4[19]))
    
    # convert to [rows, columns] structure   
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))

    # horizontally stack columns
    #dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, out_seq1, out_seq2, out_seq3, out_seq4, out_seq5, out_seq6, out_seq7))
    dataset = hstack((in_seq4, in_seq5, in_seq6,out_seq4, out_seq5, out_seq6))

    minmax = dataset_minmax(dataset)
    normalized_dataset = normalize_dataset(dataset, minmax)
    
    # define number of input and output sequences
    n_in_seq = 3
    n_out_seq = 3
        
    # choose a number of time steps
    n_steps = 15 
    
    LAT_in_rows = 4 #5*16,66ms= 83ms

    # convert into input/output
    X, y = split_sequences(normalized_dataset, n_steps, LAT_in_rows, n_in_seq, n_out_seq)

    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    print("------")
    
    return X,y

In [470]:
def prepare_training_data_euler(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    X_val_result = [] 
    y_val_result = [] 

    
    for csv in get_csv_files(dataset_path_train, "*train"):
        csv_df = load_df(csv)
        X_single, y_single = get_sequence_euler_radiant(csv_df, dataset_path_train)
        
        # split into X_single_train, X_single_test and y_single_train, y_single_test with 80/20 split
        train_size = int(len(X_single) * 0.80)
        test_size = len(X_single) - train_size
        X_train, X_val = X_single[0:train_size,:], X_single[train_size:len(X_single),:] 
        y_train, y_val =  y_single[0:train_size,:], y_single[train_size:len(y_single),:]     
        
        X_train_result.append(X_train)
        y_train_result.append(y_train)
        
        X_val_result.append(X_val)
        y_val_result.append(y_val)
        
    X_train_result = np.vstack((X_train_result))
    y_train_result = np.vstack((y_train_result))

    X_val_result = np.vstack((X_val_result))
    y_val_result = np.vstack((y_val_result))
        
    return X_train_result, X_val_result, y_train_result, y_val_result

In [471]:
######################################################################
# Init Configuration
######################################################################

config_path = 'C:/Users/weikert1/Documents/Thesis/Project_V06_26072021/Code/V06_config.toml'
cfg = toml.load(config_path)

# General 
n_in_seq = cfg['n_in_seq_euler']
n_out_seq = cfg['n_out_seq_euler']
n_steps = cfg['n_steps'] 
LAT_in_rows = cfg['LAT_in_rows'] #6*16,66ms= 64ms

# model ID --> specify this for every new model
model_id = cfg['4_2_model_ID']
model_type = cfg['4_model_type']

# Directories 
Path_dataset01_test = cfg['Path_dataset01']
Path_results_dataset01_GRU_euler_model_id = cfg['Path_results_dataset01'] + model_type + "/" + model_id  + "/" 

# Tensorflow configuration
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))



Num GPUs Available:  1


In [472]:
######################################################################
# Load Training Data
######################################################################

#pd.set_option("display.precision", 2)
X_train, X_val, y_train, y_val = prepare_training_data_euler(Path_dataset01_test)

Original y_true : [5.775161]
Shifted rad y_true : [0.10087727]
X shape is: (71406, 15, 3)
y shape is: (71406, 3)
------


In [473]:
y_train[0]

array([0.01605512, 0.99298423, 0.2367753 ])

In [443]:
def load_scaler_objects(path_scaler_objects):

    out_seq4_scaler = load(open(path_scaler_objects + 'out_seq4_scaler.pkl', 'rb'))
    out_seq5_scaler = load(open(path_scaler_objects + 'out_seq5_scaler.pkl', 'rb'))
    out_seq6_scaler = load(open(path_scaler_objects + 'out_seq6_scaler.pkl', 'rb'))
    
    out_seq4_scaler_min = out_seq4_scaler.data_min_
    out_seq5_scaler_min = out_seq5_scaler.data_min_
    out_seq6_scaler_min = out_seq6_scaler.data_min_
    
    out_seq4_scaler_max = out_seq4_scaler.data_max_
    out_seq5_scaler_max = out_seq5_scaler.data_max_
    out_seq6_scaler_max = out_seq6_scaler.data_max_
    
    return out_seq4_scaler_min, out_seq5_scaler_min, out_seq6_scaler_min, out_seq4_scaler_max, out_seq5_scaler_max, out_seq6_scaler_max
    

In [254]:
def minmax_inverse_transform(y_true, y_pred):
    path_scaler_objects = Path_dataset01_test + "Scaler_objects_euler/" 
    w_min, x_min, y_min, w_max, x_max, y_max = load_scaler_objects(path_scaler_objects)
    w1, x1, y1 = tf.unstack(y_true, axis=-1)
    w2, x2, y2 = tf.unstack(y_pred, axis=-1)
    print("shifted scaled value of w1 : " + str(w1))
    w1, w2 = (w1 - K.constant(w_min, dtype=tf.float64)) / K.constant(w_max - w_min,dtype=tf.float64) , (w2 - K.constant(w_min, dtype=tf.float64)) / K.constant(w_max - w_min , dtype=tf.float64)
    x1, x2 = (x1 - K.constant(x_min, dtype=tf.float64)) / K.constant(x_max - x_min, dtype=tf.float64) , (x2 - K.constant(x_min, dtype=tf.float64)) / K.constant(x_max - x_min, dtype=tf.float64)
    y1, y2 = (y1 - K.constant(y_min, dtype=tf.float64)) / K.constant(y_max - y_min, dtype=tf.float64) , (y2 - K.constant(y_min, dtype=tf.float64)) / K.constant(y_max - y_min, dtype=tf.float64)
    print("re-inversed value of w1 : " + str(w1))
    
    print(w1)
    y_true = tf.stack((w1, x1, y1), axis=-1)
    print(y_true)
    y_pred = tf.stack((w2, x2, y2), axis=-1)
    return y_true, y_pred

In [255]:
tf.keras.backend.set_floatx('float32')

In [490]:
y_true = y_train[1]
y_test = y_train[1000]

print(y_true)

y_true_tf = tf.convert_to_tensor(y_true, dtype= np.double)
print(y_true_tf)
y_test_tf = tf.convert_to_tensor(y_test, dtype= np.double)
print(y_test_tf)

[0.01605512 0.99298423 0.2367753 ]
tf.Tensor([0.01605512 0.99298423 0.2367753 ], shape=(3,), dtype=float64)
tf.Tensor([0.02362982 0.99185506 0.23680525], shape=(3,), dtype=float64)


In [491]:
def rotation_loss(y_true,y_pred):
    rotation_loss = tf.sqrt(tf.nn.l2_loss(y_true-y_pred))
    return rotation_loss

In [492]:
rotation_loss(y_true_tf,y_test_tf)

<tf.Tensor: shape=(), dtype=float64, numpy=0.005415354114272378>

In [493]:
y1_np = y_true_tf.numpy()
y2_np = y_test_tf.numpy()

In [494]:
y = y1_np - y2_np
y

array([-7.57470833e-03,  1.12916667e-03, -2.99444444e-05])

In [495]:
import math
y = y1_np - y2_np
print(y)

a = y[0]
print(a)
b = y[1]
c = y[2]
norm = math.sqrt(a*a + b*b + c*c)
norm

[-7.57470833e-03  1.12916667e-03 -2.99444444e-05]
-0.007574708333333333


0.0076584672334569365

In [366]:
from numpy import array
from numpy.linalg import norm

d = array([a,b,c])
norm(d)

0.18327736320134166

In [362]:
a = array([1, 2, 3])
print(a)
l2 = norm(a)
print(l2)

[1 2 3]


TypeError: 'float' object is not callable

In [482]:
from scipy.spatial.transform import Rotation as R
from scipy.linalg import logm, expm



r1 = R.from_euler('xyz', [0,0,0],degrees = True).as_matrix()
r2 = R.from_euler('xyz', [0,0,90],degrees = True).as_matrix()

print("r1")
print(r1)
print("r2")
print(r2)

angle_dif_mat = logm(np.matmul(r1,r2.transpose()))
print("Angle dif")
print(angle_dif_mat)

print("norm")
a = norm(angle_dif_mat)

print("this = expected*sqrt(2)")
print("expected = pi/2")

r1
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
r2
[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]
Angle dif
[[ 0.00000000e+00  1.57079633e+00  0.00000000e+00]
 [-1.57079633e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16]]
norm


TypeError: 'float' object is not callable